#  Spatial Filtering

# Question 1

In [1]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy import misc
from numpy.linalg import inv
from PIL import Image, ImageDraw
import matplotlib.patches as patches

Using matplotlib backend: MacOSX


In [2]:
def convolution(img,kernel=0,kernel_type='none'):
    output = np.zeros_like(img)           
    # Add zero padding to the input image
    kernel_x = kernel.shape[0]
    kernel_y = kernel.shape[1]
    image_padded = np.zeros((img.shape[0] + (kernel_x-1) , img.shape[1] + kernel_y-1))   
    image_padded[1:-1, 1:-1] = img
    #print(image_padded)
    if kernel_type == 'median':
        for x in range(kernel_x//2,img.shape[0]+kernel_x//2):     
            for y in range(kernel_y//2,img.shape[1]+kernel_y//2):
               # print(x-kernel_x//2,y-kernel_y//2)
                # element wise multiplication of the image pixels and kernel
                array = image_padded[x-kernel_x//2:x+kernel_x//2+1,y-kernel_y//2:y+kernel_y//2+1]
                output[x-kernel_x//2,y-kernel_y//2] = np.median(array)
    else:        
        for x in range(kernel_x//2,img.shape[0]+kernel_x//2):     
            for y in range(kernel_y//2,img.shape[1]+kernel_y//2):
               # print(x-kernel_x//2,y-kernel_y//2)
                # element wise multiplication of the image pixels and kernel
                output[x-kernel_x//2,y-kernel_y//2]=int(((kernel*image_padded[x-kernel_x//2:x+kernel_x//2+1,y-kernel_y//2:y+kernel_y//2+1]).sum()))        
    return output

In [3]:
def imnoise(img,noise_type,amt=0):
    if(noise_type == 'gaussian'):
        row,col = img.shape
        mean = 5
        var = 15
        sigma = var**0.5
        gauss_noise = np.random.normal(mean,sigma,(row,col))
        gauss_noise = gauss_noise.reshape(row,col)
        noisy_img = img + gauss_noise
        return noisy_img
    elif noise_type == "s&p":
        row,col= img.shape
        s_vs_p = 0.5
        amount = amt
        out = np.copy(img)
      # Salt mode
        num_salt = np.ceil((amount/2) * img.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
                  for i in img.shape]
        out[coords] = 255

      # Pepper mode
        num_pepper = np.ceil((amount/2)* img.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in img.shape]
        out[coords] = 0
        return out
    else:
        return

In [4]:
img = misc.imread('lena.jpg',mode="L")
kernel = np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])
noise_remove_kernel = np.array([[1,1,1],[1,1,1],[1,1,1]])*(1/9) #mean filter
given_kernel = np.reshape(kernel,(3,3))
kernel = np.zeros(given_kernel.shape)

#flipping kernel
for i in range(0,kernel.shape[0]):
    for j in range(0,kernel.shape[1]):
        kernel[i,j] = given_kernel[-i+(given_kernel.shape[0]-1),-j+(given_kernel.shape[1]-1)]
# image after adding noise
density = [0.05,0.1,0.2 ]
for dens in density:
    noisy_img  = imnoise(img,'s&p',dens)
    output_noisy = convolution(noisy_img,kernel,'median')
    #noisy image
    plt.figure()
    plt.subplot(121)
    plt.imshow(noisy_img)
    plt.gray()
    plt.show()

    #output of noisy image convolution
    plt.subplot(122)
    plt.imshow(output_noisy)
    plt.gray()
    plt.show()

# Question 2

In [5]:
img = misc.imread('lena.jpg',mode="L")
img1 = np.copy(img)
plt.figure()
plt.gray()
plt.imshow(img1)
noise_remove_kernel = np.array([[1,2,1],[0,0,0],[-1,-2,-1]]) # sobel filter
#noise_remove_kernel = np.array([[1,1,1],[1,1,1],[1,1,1]])*(1/9) #mean filter
#noise_remove_kernel = np.array([[0,-1,0],[-1,4,-1],[0,-1,0]]) #laplacian filter
#noise_remove_kernel = np.array([[-1,2,1],[0,1,2],[-1,1,0]])

rows,cols = img.shape
r,c = noise_remove_kernel.shape

#shifting low frequencies to middle 
for i in range(0,rows):
     for j in range(0,cols):
        img1[i,j] = img1[i,j]*((-1)**(i+j))
# -------------------------------------------------------------------------------------        
#padding of image and kernel

img_padded = np.zeros((rows+r-1,cols+c-1))
#print("a_tempo shape ",a_temp.shape)
kernel_padded = np.zeros_like(img_padded)
print(img_padded.shape,kernel_padded.shape)

img_padded[1:-1,1:-1] = img1
m,n = kernel_padded.shape
m = m//2
n = n//2
print(m,n)
#kernel_padded[m-1:m-1+r,n-1:n-1+c]= noise_remove_kernel
kernel_padded[0:r,0:c] = noise_remove_kernel
# for i in range(0,kernel_padded.shape[0]):
#      for j in range(0,kernel_padded.shape[1]):
#         kernel_padded[i,j] = kernel_padded[i,j]*((-1)**(i+j))
#kernel_padded[0:r,0:c] = noise_remove_kernel

#--------------------------------------------------------------------------



# 2D FFT of  image
img_padded = np.fft.fft(img_padded,axis=1)
img_padded = np.fft.fft(img_padded,axis=0)

# 2D fft of kernel

kernel_padded= np.fft.fft(kernel_padded,axis=1)
kernel_padded = np.fft.fft(kernel_padded,axis=0)

plt.figure()
plt.gray()
plt.subplot(121)
plt.imshow(img_padded.real)
plt.subplot(122)
plt.imshow(kernel_padded.real)
output = img_padded*kernel_padded

# ifft

output = np.fft.ifft(output,axis=1)
output = np.fft.ifft(output,axis=0)
for i in range(0,output.shape[0]):
     for j in range(0,output.shape[1]):
        output[i,j] = output[i,j]*((-1)**(i+j))
plt.figure()
plt.imshow(output.real)
plt.gray()
plt.show() 


(202, 202) (202, 202)
101 101


# Question 3

In [6]:
def cross_cor_n(img,mask):
    M = mask.shape[0]
    N = mask.shape[1]
    offset_row = int(M/2)
    offset_col = int(N/2)
    new_img = np.zeros(shape=(img.shape[0]+M,img.shape[1] + N),dtype=np.integer)
    new_img[M-1 : -1,N-1 : -1] = img
    new_conv_img = np.zeros(shape=(img.shape[0],img.shape[1] ),dtype=np.float)
    mask_mean = np.mean(mask)
    mask_norm = (np.sum(np.square(mask - mask_mean)))**0.5
    for i in range(new_conv_img.shape[0]):
        for j in range(new_conv_img.shape[1]):

            img_mean = np.mean(new_img[i:i+M , j:j+N])
            img_norm = (np.sum(np.square(new_img[i:i+M , j:j+N] - img_mean)))**0.5
            #Variance = 0 condition to be handled
            if(img_norm != 0):
                new_conv_img[i,j] = ((np.sum((mask - mask_mean) * (new_img[i:i+M , j:j+N] - img_mean)))/(mask_norm*img_norm))
            else:
                new_conv_img[i,j] = 0
                
    return (new_conv_img)

In [7]:
img1 = misc.imread('lab4text.png',mode="L")
img2 = misc.imread('and.png',mode="L")
result = cross_cor_n(img1,img2)
max1 = np.max(result)
ma_i = []
for i in range(img2.shape[0],result.shape[0]-1):
    for j in range(img2.shape[1],result.shape[1]-1):
        if(result[i,j] >= max1*0.9):
            ma_i.append((i,j))
print(len(ma_i))
#fig1 = plt.figure(figsize=(20,20))
im = Image.open('lab4text.png')
draw = ImageDraw.Draw(im)
for i in range(int(len(ma_i))):
    #rect = (patches.Rectangle((ma_i[i][0],ma_i[i][1]+img2.shape[1]),img2.shape[1],img2.shape[0],linewidth=1,edgecolor='r',facecolor='none'))
    draw.rectangle([ma_i[i][1]-img2.shape[1], ma_i[i][0]-img2.shape[0],ma_i[i][1] ,ma_i[i][0] ], outline=(200, 0, 100, 255))
im.show()

6
